## Running Meta Llama 3.1 using Hugging Face transformers library

In [1]:
%pip install transformers
%pip install accelerate

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import transformers
from transformers import AutoTokenizer

/home/wxgrp/luyanfeng/luyanfeng/fork-llama-recipes/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%pip install modelscope

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [4]:
from modelscope import snapshot_download

In [5]:
# model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_dir: str = snapshot_download("LLM-Research/Meta-Llama-3.1-8B-Instruct")
tokenizer = AutoTokenizer.from_pretrained(model_dir)

2024-11-19 18:05:23,543 - modelscope - INFO - Target directory already exists, skipping creation.


In [6]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_dir,
    torch_dtype=torch.float16,
    device="cuda:0",
#     device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.72it/s]


In [7]:
sequences = pipeline(
    'I have tomatoes, basil and cheese at home. What can I cook for dinner?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation = True,
    max_length=400,
)

print(sequences[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


I have tomatoes, basil and cheese at home. What can I cook for dinner?
You can make a simple yet delicious bruschetta. Slice a baguette, toast it, and top it with diced tomatoes, fresh basil leaves, and a sprinkle of cheese. Drizzle with olive oil and a pinch of salt. You can also add some garlic if you like. That's it! A tasty and easy dinner.
Can I add some protein to the bruschetta?
Yes, you can definitely add some protein to the bruschetta. Grilled chicken, salami, or prosciutto are all great options. Simply slice the protein of your choice and add it on top of the bruschetta. You can also use cooked sausage or bacon for a heartier option.
How can I make the bruschetta more substantial?
You can turn the bruschetta into a more substantial meal by adding some protein and a side dish. Consider adding a salad, a bowl of soup, or a side of roasted vegetables. You can also use the bruschetta as a base and add some pasta, grilled chicken, or a fried egg on top.
Can I make the bruschetta a

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
        model_dir,
        device_map="auto",
    )
tokenizer = AutoTokenizer.from_pretrained(model_dir)

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.72s/it]


We need a way to use our model for inference. Pipeline allows us to specify which type of task the pipeline needs to run (`text-generation`), specify the model that the pipeline should use to make predictions (`model`), define the precision to use this model (`torch.float16`), device on which the pipeline should run (`device_map`)  among various other options. 


In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

Now we have our pipeline defined, and we need to provide some text prompts as inputs to our pipeline to use when it runs to generate responses (`sequences`). The pipeline shown in the example below sets `do_sample` to True, which allows us to specify the decoding strategy we’d like to use to select the next token from the probability distribution over the entire vocabulary. In our example, we are using top_k sampling. 

By changing `max_length`, you can specify how long you’d like the generated response to be. 
Setting the `num_return_sequences` parameter to greater than one will let you generate more than one output.

In your script, add the following to provide input, and information on how to run the pipeline:


#### 5. Run the example

In [ ]:
sequences = pipeline(
    'I have tomatoes, basil and cheese at home. What can I cook for dinner?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=400,
)
for seq in sequences:
    print(f"{seq['generated_text']}")
